In [ ]:
from googleapiclient.discovery import build
import json
import datetime
from dateutil.relativedelta import relativedelta

# チャンネルIDを別ファイルから取得
import src.channelId as cid
import src.videoId as vid

# 必要モジュールのインポート
import os
from dotenv import load_dotenv

# .envファイルの内容を読み込見込む
load_dotenv()

# API情報
API_KEY = os.environ['API_KEY']
API_NAME = 'youtube'
API_VERSION = 'v3'

video_id_list = [] # 動画IDをリストとして取得
video_info_list = [] # APIで取得した動画情報を格納する配列

In [ ]:
def getVideoThumbnailList(page_token, start_date, end_date, channel_id=cid.CHANNEL_ID[0]):
    youtube = build(
        API_NAME,
        API_VERSION,
        developerKey = API_KEY
    )

    # 以下で指定した結果を格納する変数
    search_response = youtube.search().list(
        channelId = channel_id,
        part = 'snippet',
        # fields='items(id,snippet(title,description,publishedAt),contentDetails(duration),statistics(viewCount,likeCount,dislikeCount,commentCount))',
        type = 'video',
        maxResults = 50,                # search()1回で取得できる上限が50件（指定できるのは 0~50）
        publishedAfter = start_date,    # この日から
        publishedBefore = end_date,     # この日まで
        pageToken = page_token
    ).execute()


    # この回で取得できる総件数
    print(search_response["pageInfo"]["totalResults"])


    for search_result in search_response.get("items", []):

        # 取得した情報の分類が”動画”の場合（e.g. チャンネル："youtube#channel" / プレイリスト："youtube#playlist"）
        if search_result["id"]["kind"] == "youtube#video":

            # 選択したデータから辞書を作成する
            d = {'id': search_result["id"]["videoId"],}

            # 動画IDをリストに追加
            video_id_list.append(search_result["id"]["videoId"])


    # 動画IDを表示
    print(video_id_list)


    try:
        nextPage_token =  search_response["nextPage_Token"]
        # print(nextPage_token)
        getVideoThumbnailList(nextPage_token, start_date, end_date)
    except:
        return

In [ ]:
# 指定した日付から1ヶ月分のデータを取得
dt = datetime.datetime(2022, 1, 1, 0, 0)

# 指定した日時をisoフォーマットで表示
print(dt.isoformat())

# month=1の数字を変えると〜ヶ月分のデータ取得
# getVideoIdFromChannelId('', dt.isoformat()+'Z', (dt + relativedelta(months=1)).isoformat()+'Z')
getVideoThumbnailList('', dt.isoformat()+'Z', (dt + relativedelta(months=1)).isoformat()+'Z')

In [ ]:
# 選択した動画情報をjsonファイルに書き込む
with open("./result/result.json", mode="w", encoding="utf-8") as f_result:
    # 辞書をjsonファイルとして保存 json.dump(辞書, ファイルオブジェクト, ensure_ascii, インデント)
    json.dump(video_info_list, f_result, ensure_ascii=False, indent=2)